<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
from __future__ import absolute_import, division, print_function
import os
import numpy as np
from __future__ import absolute_import, division, print_function
from __future__ import absolute_import, division, print_function
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.imdb.load_data(
    path='imdb.npz',
    num_words=None,
    skip_top=0,
    maxlen=None,
    seed=113,
    start_char=1,
    oov_char=2,
    index_from=3
)
unique, counts = np.unique(y_train, return_counts=True)
positive_sample_count = counts[1]
idx = np.argwhere(y_train>0) # Select positive comment's index in training data
np.random.seed(seed=100) # use seed to ensure selected records are always same
np.random.shuffle(idx) #Shuffle it at random
FRAC = 0.25
idxs = idx[:int(len(idx)*FRAC)]
y_trains = y_train[idxs]
x_trains = x_train[idxs]
idxn = np.argwhere(y_train==0)
x_train0 = x_train[idxn]
y_train0 = y_train[idxn]
temp_idx = np.arange(0, len(idxs), 1)
temp_idx_multi = np.tile(temp_idx, (1, 10)).T # shape is (31250, 1)
np.random.seed(seed=200) # rerun from temp_idx_multi definition cell to ensure same result
np.random.shuffle(temp_idx_multi)
sample_idx = temp_idx_multi[:positive_sample_count].squeeze(1)
x_train[sample_idx] # no point of using just the positive, since we are going by indeces anyway
x_trains_os = x_train[sample_idx] # again, we are using indeces
y_trains_os = y_train[sample_idx]
x_train_assembled = np.concatenate((x_train0, x_trains_os), axis = None)
y_train_assembled = np.concatenate((y_train0, y_trains_os), axis = None)
shuffled_idx = np.arange(0, len(y_train_assembled), 1)
np.random.seed(seed=300)
np.random.shuffle(shuffled_idx)
print(shuffled_idx.shape)
shuffled_idx
x_train_assembled_shuffled = x_train_assembled[shuffled_idx]
y_train_assembled_shuffled = y_train_assembled[shuffled_idx]
print(x_train_assembled_shuffled.shape, y_train_assembled_shuffled.shape)
word_index = tf.keras.datasets.imdb.get_word_index()
word_index = {k:(v+3) for k,v in word_index.items()} 
word_index["<PAD>"] = 0
word_index["<START>"] = 1
word_index["<UNK>"] = 2  # unknown
word_index["<UNUSED>"] = 3
index_word = dict([(value, key) for (key, value) in word_index.items()])
def decode_review(encoded_array):
    return ' '.join([index_word.get(i, '?') for i in encoded_array])
train_data = tf.keras.preprocessing.sequence.pad_sequences(x_train_assembled_shuffled,
                                                        value=word_index["<PAD>"],
                                                        padding='pre',
                                                        maxlen=256)
test_data = tf.keras.preprocessing.sequence.pad_sequences(x_test,
                                                       value=word_index["<PAD>"],
                                                       padding='pre',
                                                       maxlen=256)
vocab_size = len(word_index)
MAX_SENTENCE_LENGTH=256
EMBEDDING_SIZE=16
HIDDEN_LAYER_SIZE=64
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, 64),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.summary()
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
FRAC = 0.8 # fraction of training data used for training. Remaining is for cross validation.
idx = np.arange(len(train_data))
np.random.seed(seed=400)
np.random.shuffle(idx)
idxs = idx[:round(len(idx)*FRAC)] # Select random 80% for training data
partial_x_train = train_data[idxs]
partial_y_train = y_train_assembled[idxs]
x_val = np.delete(train_data, idxs.tolist(), axis=0) # select remaining as cross validation data
y_val = np.delete(y_train_assembled, idxs.tolist(), axis=0)
print(partial_x_train.shape, partial_y_train.shape)
print(x_val.shape, y_val.shape)
history = model.fit(partial_x_train,
                    partial_y_train,
                    epochs=40,
                    batch_size=512,
                    validation_data=(x_val, y_val),
                    verbose=1)
predicted = model.predict(test_data)
predicted
predicted[predicted > 0.5] = 1
predicted[predicted <= 0.5] = 0
predictedf = predicted.flatten().astype(int)
import pandas as pd
df3 = pd.DataFrame(data=predictedf, columns=['predicted'])
refdf = pd.DataFrame(data=y_test, columns=['actual'])
y_actu = pd.Series(refdf['actual'], name='ACTUAL')
y_pred = pd.Series(df3['predicted'], name='PREDICTED')
predicted_results = y_pred.tolist()
truth = y_actu.tolist()
dl_confusion = pd.crosstab(y_actu, y_pred, rownames=['Actual'], colnames=['Predicted'], margins=True)
dl_confusion
from sklearn.metrics import classification_report
report = classification_report(truth, predicted_results)
print(report)


(25000,)
(25000,) (25000,)
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 64)          5669632   
_________________________________________________________________
bidirectional (Bidirectional (None, 256)               197632    
_________________________________________________________________
dense (Dense)                (None, 64)                16448     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 5,883,777
Trainable params: 5,883,777
Non-trainable params: 0
_________________________________________________________________
(20000, 256) (20000,)
(5000, 256) (5000,)
Epoch 1/40
40/40 [==============================] - 52s 1s/step - loss: 0.6197 - accuracy: 0.7255 - val_loss: 0.5500 - val_accuracy: 0.7614
Epoch 2/40
40/40 [=================